## Loend nimeüksustest test50

In [119]:
from estnltk import Text
import sqlite3
import csv
from collections import Counter
import itertools

Alustuseks loetakse andmebaasist sisse kõik isiku- ja organisatsiooninimed

In [174]:
entities_PER = []
entities_ORG = []

In [175]:
con = sqlite3.connect("media_data_2.db")
cur = con.cursor()

In [176]:
for row in cur.execute("SELECT PER FROM named_entities where PER IS NOT NULL"):
    entities_PER.append(row[0])

In [177]:
for row in cur.execute("SELECT ORG FROM named_entities where ORG IS NOT NULL"):
    entities_ORG.append(row[0])

In [178]:
con.close()

Vaatame saadud järjendeid:

In [179]:
print(entities_PER[:20])

['hans', 'eerik', 'andrus ansip', 'meelis', 'olev esula', 'hans', 'vadim', 'juskin', 'margus', 'vadim', 'sven', 'sven', 'siim kallas', 'deniss borodits', 'jukuminek', 'borodits', 'jüri', 'eiki', 'jüri', 'hans']


In [180]:
print(entities_ORG[:20])

['rein', 'h', 'taska', 'india', 'rein', 'h', 'vadim belov', 'pbk', 'delfi', 'tallinn ülikool', 'el', 'delfi', 'ccbe', 'kapo', 'irl roheline', 'sots', 'keskerakond', 'etv', 'demokraat obama', 'err']


Esiteks eemaldan nimede hulgast need nimed, mis viitavad saatejuhtidele (sageli mainitakse saate sissejuhatuses ja lõpus):

In [181]:
speakers = []

In [182]:
con = sqlite3.connect("media_data_2.db")
cur = con.cursor()

In [183]:
for row in cur.execute("SELECT DISTINCT person FROM lines"):
    speakers.append(row[0])

In [184]:
con.close()

In [185]:
print(speakers)

['Tundmatu3', 'Tundmatu1', 'Tundmatu2', 'Tundmatu4', 'Tundmatu5', 'Tundmatu6', 'K1', 'K2', 'K3', 'K4', 'K5', 'K6', 'K7', 'Erakordset looduskaunites kohtades erakordselt efekt siis keskkonnas näiteks näiteks Monte', 'K8', 'Heldur Meerits', 'Ignar Fjuk', 'Marek Strandberg', 'Janek Luts', 'Priit Hõbemägi', 'K9', 'Hõbemägi Priit', 'Astok Hannes', 'Ruussaar Ainar', 'Ainar Ruussaar', 'K06', 'K02', 'K03', 'K07', 'K01', 'K04', 'K05', 'Artur Talvik']


In [186]:
speakers_lower = [speaker.lower() for speaker in speakers]

In [187]:
entities_PER_cleaned = [name for name in entities_PER if name not in speakers_to_remove]

Esialgsed isiku- ja organisatsiooninimede sagedusloendid:

In [188]:
frequencies_PER = Counter(entities_PER_cleaned)
frequencies_ORG = Counter(entities_ORG)

In [190]:
print('PER' + '\t' + 'ORG')
for (per, org) in itertools.zip_longest(frequencies_PER.most_common(10), frequencies_ORG.most_common(10)):
    print(per, org)

PER	ORG
('edgar savisaar', 99) ('keskerakond', 215)
('jüri ratas', 61) ('reformierakond', 197)
('siim kallas', 37) ('riigikogu', 157)
('savisaar', 33) ('irl', 103)
('indrek', 32) ('nato', 35)
('mart luik', 30) ('rein', 32)
('andrus ansip', 27) ('postimees', 29)
('ansip', 27) ('keskpäevatund', 24)
('reinsalu', 26) ('eesti pank', 22)
('rail baltic', 26) ('eas', 22)


Esialgsest sagedusloendist on näha, et paljudele täispikkadele isikunimedele esineb ka vasteid kas ainult eesnime või ainult perekonnanime näol (nt Edgar Savisaar ja Savisaar). Samas ei pruugi selliseid vasteid esineda kõigile täispikkadele isikunimedele käesolevas loendis. Et hilisemat nimeolemite otsingut parandada, tükeldan kõik täispikad nimed ees- ja perekonnanimedeks. Järgneva lähenemise puuduseks on see, et ei püüta eristada ees- ja perenimest koosnevaid üksuseid kahest (või enamast) tühikuga eraldatud eesnimest koosnevatest üksustest (nt Toomas Hendrik). Mitmest eesnimest koosnevaid üksuseid oleks võimalik eristada osalise edukusega, kui tekstis leidub ka koos perekonnanimega täispikk variant, mille vastu eesnimesid võrrelda. Organisatsiooninimede peal seda lähenemist ei kasutata, sest pole alust arvata, et isikunimede kasutustavad organisatsiooninimedele laieneks. Pigem on sagedasem organisatsiooninimede puhul täispika nime asendamine lühendi/akronüümiga (nt Kaitsepolitseiamet -> KAPO). Siin praegu ei püüta lühendite/akronüümide võrdsustamist täispikkade nimedega lahendada.

In [191]:
entities_splitted = [name.rsplit(' ', 1) for name in entities_PER_cleaned if len(name.split(' ')) > 1]

In [192]:
entities_unnested = [name for entity in entities_splitted for name in entity]

Saadud järjend kleebitakse varasema isikunimede järjendi lõppu:

In [193]:
entities_PER_cleaned = entities_PER_cleaned + entities_unnested

In [194]:
#Uus sagedusloend isikunimeüksustele
frequencies_PER = Counter(entities_PER_cleaned)

Kuna praegusel kujul on jäänud alles ka siiski vigaseid nimesid, mis võivad hiljem probleeme tekitada (nt "ja"), peab eemaldama haruldased juhud enne uut nimeüksuste märgendamist. Hetkel on analüüsitavaid tekste 50, alampiiriks valitakse hetkel üsna suvaliselt päris väike esinemissagedus. Alampiiri võib aga määrata ka statistika abil.

In [195]:
names_to_remove = []

for name in frequencies_PER.keys():
    if frequencies_PER[name] < 10:
        names_to_remove.append(name)
for name in frequencies_ORG.keys():
    if frequencies_ORG[name] < 10:
        names_to_remove.append(name)

In [196]:
print(names_to_remove[:20])

['hans', 'eerik', 'olev esula', 'vadim', 'juskin', 'deniss borodits', 'jukuminek', 'borodits', 'kerviel', 'lasna', 'heimar lenk', 'häkkinen', 'molly', 'raimo', 'evelin', 'baran', 'batory', 'toomas henrik', 'hannes hanso', 'clyde']


In [197]:
entities_PER_cleaned = [name for name in entities_PER_cleaned if name not in names_to_remove]
entities_ORG_cleaned = [name for name in entities_ORG if name not in names_to_remove]

In [198]:
unique_PER = set(entities_PER_cleaned)
unique_ORG = set(entities_ORG_cleaned)

Saadud loendid talletatakse csv-failis:

In [199]:
with open('entity_phrases.csv', 'w', newline='', encoding='UTF-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['phrase', 'value'])
    writer.writeheader()
    header2 = {'phrase': 'callable', 'value': 'string'}
    writer.writerow(header2)
    split_names_PER = [name.split() for name in unique_PER]
    split_names_ORG = [name.split() for name in unique_ORG]
    for name in split_names_PER:
        if len(name) > 1:
            writer.writerow({'phrase': ','.join(["'" + part + "'" for part in name]), 'value': 'PER'})
        else:
            writer.writerow({'phrase': "'" + name[0] + "'" + ",", 'value': 'PER'})
    for name in split_names_ORG:
        if len(name) > 1:
            writer.writerow({'phrase': ','.join(["'" + part + "'" for part in name]), 'value': 'ORG'})
        else:
            writer.writerow({'phrase': "'" + name[0] + "'" + ",", 'value': 'ORG'})

In [171]:
with open('entity_phrases.csv', encoding='UTF-8') as csv_file:
    rows = []
    reader = csv.DictReader(csv_file)
    for row in reader:
        rows.append(row)

In [172]:
print(rows[:10])

[{'phrase': 'callable', 'value': 'string'}, {'phrase': "'agu',", 'value': 'PER'}, {'phrase': "'jaanson',", 'value': 'PER'}, {'phrase': "'oudekki',", 'value': 'PER'}, {'phrase': "'vjatšeslav',", 'value': 'PER'}, {'phrase': "'andrus',", 'value': 'PER'}, {'phrase': "'ardo',", 'value': 'PER'}, {'phrase': "'soonvald',", 'value': 'PER'}, {'phrase': "'riisalu',", 'value': 'PER'}, {'phrase': "'arto',", 'value': 'PER'}]


Hetkel ei ole isikunimeüksuste juurde märgitud, kas tegemist on poliitikuga või mitte, kuid seda on võimalik teha käsitsi.